In [1]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [2]:
sys.path.append('../..')

In [3]:
from lung_cancer.dataset import FilesIndex
from lung_cancer import CTImagesMaskedBatch as CTIMB

### Global index

In [4]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [5]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [6]:
ind = FilesIndex(path=DIR_LUNA, no_ext=True)

### Create batch by subindexing

In [7]:
batch_masked = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))
batch_masked_2 = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))

### Test actions

* check load

In [8]:
batch_masked.load(fmt='raw')
batch_masked_2.load(fmt='raw')

In [9]:
batch_copy = deepcopy(batch_masked)

* load info about nodules in batches

In [10]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')
nodules.head(2)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708


In [11]:
batch_masked = batch_masked.fetch_nodules_info(nodules=nodules)
batch_masked_2 = batch_masked_2.fetch_nodules_info(nodules=nodules)

* create mask in one of two batches before resize

In [12]:
batch_masked.create_mask()

* check resize

In [13]:
batch_masked = batch_masked.resize(shape=(128, 256, 256), n_workers=6)

In [14]:
batch_masked_2 = batch_masked_2.resize(shape=(128, 256, 256), n_workers=6)

* create mask in the second batch after resize

In [15]:
batch_masked_2 = batch_masked_2.create_mask()

In [16]:
batch_masked_2.indices

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'],
      dtype='<U101')

In [17]:
batch_masked.indices

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'],
      dtype='<U101')

In [18]:
batch_copy.fetch_nodules_info(nodules)
batch_copy.create_mask()

### Compare

In [19]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [20]:
n_pat = 0

In [21]:
ind_npat = batch_masked.indices[1]

In [22]:
interact(lambda height: plot_arr_slices(height, 
                                        batch_masked.get(ind_npat, 'images'),
                                        batch_masked_2.get(ind_npat, 'masks'),
                                        batch_copy.get(ind_npat, 'masks')),
         height=(0.01, 0.99, 0.01))



<function __main__.<lambda>>

### Sample nodules from batches

In [49]:
nods_batch = batch_masked.sample_nodules(batch_size=10, nodule_size=(32, 64, 64), share=0.7,
                                         variance=[100, 400, 400])

In [70]:
nods_batch.dump(dst=DIR_DUMP)

selector_events.py[LINE:53]#DEBUG    [2017-07-11 14:14:10,158]  Using selector: EpollSelector


In [75]:
nods_from_dump = CTIMB(FilesIndex(path=DIR_DUMP + '/*', dirs=True))
nods_from_dump.load(fmt='blosc')

In [76]:
nnod = 6
ix_nnod = nods_batch.indices[nnod]

In [81]:
interact(lambda height: plot_arr_slices(height, 
                                        nods_batch.get(ix_nnod, 'images'),
                                        nods_batch.get(ix_nnod, 'masks'),
                                        nods_from_dump.get(ix_nnod, 'masks')),
                                        
         height=(0.01, 0.99, 0.01))

<function __main__.<lambda>>